In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)1
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutageDAQ import OutageDataInfo as ODI
from OutageMdlrPrep import OutageMdlrPrep
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
# Functions moved from _METHOD notebook to OutageMdlrPrep class
# get_active_SNs_for_xfmrs_OLD
# add_xfmr_active_SNs_to_rcpo_df_OLD
# build_rcpo_df_norm_by_xfmr_active_nSNs_OLD
# reset_index_and_identify_cols_to_merge
# merge_rcpo_and_df
# get_active_SNs_for_xfmrs
# get_active_SNs_for_xfmrs_in_rcpo_df
# get_active_SNs_for_xfmrs_in_rcpo_df_v2
# add_xfmr_active_SNs_to_rcpo_df
# build_rcpo_df_norm_by_xfmr_active_nSNs
# get_outg_time_infos_df
# build_reason_counts_per_outage_from_csvs_v0
# check_end_events_merge_with_mp
# perform_build_RCPX_from_csvs_prereqs_v0
# find_all_cols_with_gpd_for_sql_appendix
# identify_ede_cols_of_interest_to_update_andor_drop
# perform_build_RCPX_from_csvs_prereqs
# drop_gpd_for_sql_appendix_from_all_cols
# drop_gpd_for_sql_appendix_from_all_index_names
# build_reason_counts_per_outage_from_csvs


In [ ]:
#-------------------------
run_date = '20240906' # Date of data acquisition
#-------------------------
date_0   = '2023-04-01' # Lower limit for end events
date_1   = '2024-08-31' # Upper limit for end events
#-------------------------
# dataset  = 'outg'
dataset  = 'otbl'
# dataset  = 'prbl'
#-------------------------
save_dfs_to_pkl   = True
read_dfs_from_pkl = False

In [ ]:
# #--------------------------------------------------
# outg_mdlr_prep = OutageMdlrPrep(
#     dataset         = dataset, 
#     run_date        = run_date, 
#     date_0          = date_0, 
#     date_1          = date_1, 
#     data_evs_sum_vw = False, 
#     data_base_dir   = None, 
#     verbose         = True
# )

In [ ]:
#--------------------------------------------------
ODI.assert_dataset(dataset)
#-------------------------
date_pd_subdir = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
data_base_dir  = os.path.join(
    Utilities.get_local_data_dir(), 
    r'dovs_and_end_events_data', 
    run_date, 
    date_pd_subdir, 
    ODI.get_subdir(dataset)
)
#-------------------------
assert(os.path.isdir(data_base_dir))
#-------------------------
files_dir    = os.path.join(data_base_dir, 'EndEvents')
naming_tag   = ODI.get_naming_tag(dataset)
is_no_outage = ODI.get_is_no_outage(dataset)
#-------------------------
assert(os.path.isdir(files_dir))
#-------------------------
print(f'data_base_dir = {data_base_dir}')
print(f'files_dir     = {files_dir}')
print(f'naming_tag    = {naming_tag}')
print(f'is_no_outage  = {is_no_outage}')

In [ ]:
#--------------------------------------------------
file_path_glob                = r'end_events_[0-9]*.csv'
file_path_regex               = None

assert_all_cols_equal         = False #Seems new EndEvents have curr_acct_cls_cd as well...
include_normalize_by_nSNs     = True
inclue_zero_counts            = True
return_multiindex_outg_reason = False
return_normalized_separately  = False

# 0-31, 1-6, 6-11, 11-16, 16-21, 21-26, 26-31
days_min_outg_td_window=6
days_max_outg_td_window=11

xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']

# In the past, the mp_df data (which was saved during DAQ) were needed for the trsf_pole_nb information
# However, these data are now typically saved with the end_events data themselves (in the csvs), so 
#   mp_df is rarely needed anymore
mp_df_needed_to_build_rcpx_from_csvs = False

In [ ]:
#--------------------------------------------------
if is_no_outage:
    batch_size      = 100
    grp_by_col      = ['trsf_pole_nb', 'no_outg_rec_nb']
    outg_rec_nb_col = 'no_outg_rec_nb'
else:
    batch_size      = 1000
    grp_by_col      = ['outg_rec_nb', 'trsf_pole_nb']
    outg_rec_nb_col = 'outg_rec_nb'
#-------------------------
if include_normalize_by_nSNs and not return_normalized_separately:
    normalize_by_nSNs_included=True
else:
    normalize_by_nSNs_included=False

#--------------------------------------------------
assert(save_dfs_to_pkl+read_dfs_from_pkl <=1) # Should never both read and write!
#--------------------------------------------------
# Currently, expecting grp_by_col to be 'outg_rec_nb', 'trsf_pole_nb', or ['outg_rec_nb', 'trsf_pole_nb']
#   Actually, 'outg_rec_nb' will probably not be run again, instead will likely always be ['outg_rec_nb', 'trsf_pole_nb']
assert(
    grp_by_col=='outg_rec_nb' or 
    grp_by_col==['outg_rec_nb', 'trsf_pole_nb'] or 
    grp_by_col=='trsf_pole_nb' or 
    grp_by_col==['trsf_pole_nb', 'no_outg_rec_nb']
)

# Not possible for have outg_rec_nb for no outage case!
if is_no_outage:
    assert(grp_by_col!='outg_rec_nb' and grp_by_col!=('outg_rec_nb', 'trsf_pole_nb'))
#--------------------------------------------------
save_subdir_pkls = 'rcpo_dfs'
if   grp_by_col == ['outg_rec_nb', 'trsf_pole_nb']:
    save_subdir_pkls += '_GRP_BY_OUTG_AND_XFMR'
elif grp_by_col == 'trsf_pole_nb':
    save_subdir_pkls += '_GRP_BY_XFMR'
elif grp_by_col == 'outg_rec_nb':
    save_subdir_pkls += '_GRP_BY_OUTG'
elif grp_by_col == ['trsf_pole_nb', 'no_outg_rec_nb']:
    save_subdir_pkls += '_GRP_BY_NO_OUTG_AND_XFMR'
else:
    assert(0)
#-----
save_dir_base_pkls = os.path.join(data_base_dir, save_subdir_pkls)
save_dir_pkls      = os.path.join(save_dir_base_pkls, f'outg_td_window_{days_min_outg_td_window}_to_{days_max_outg_td_window}_days')
#-----
if save_dfs_to_pkl and not os.path.exists(save_dir_pkls):
    os.makedirs(save_dir_pkls)

In [ ]:
# TODO NEED TO BE AUTOMATED
if is_no_outage:
    outg_rec_nb_idx_lvl      =-1
    trsf_pole_nbs_idx_lvl    = 0
    
    trsf_pole_nbs_loc        = ('index', 'trsf_pole_nb')
    rcpo_df_to_time_infos_on = [('index', 'trsf_pole_nb'), ('index', 'no_outg_rec_nb')]
    time_infos_to_rcpo_df_on = [('index', 'trsf_pole_nb'), ('index', 'no_outg_rec_nb')]
    
    rcpo_df_to_PNs_on        = ['index']
    PNs_to_rcpo_df_on        = ['index']
    how                      = 'left'
else:
    outg_rec_nb_idx_lvl      = 0
    trsf_pole_nbs_idx_lvl    = 1
    
    trsf_pole_nbs_loc        = ('index', 'trsf_pole_nb')
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')]
    time_infos_to_rcpo_df_on = ['index']
    
    rcpo_df_to_PNs_on        = [('index', 'trsf_pole_nb')]
    PNs_to_rcpo_df_on        = ['index']
    how                      = 'left'

# Grab mp_df if needed

In [ ]:
if mp_df_needed_to_build_rcpx_from_csvs:
    mp_df_path      = os.path.join(data_base_dir, r'df_mp_outg_full.pkl')
    mp_df           = pd.read_pickle(mp_df_path)
    merge_on_mp     = ['mfr_devc_ser_nbr', 'prem_nb', 'OUTG_REC_NB']
    mp_df_cols      = dict(
        serial_number_col = 'mfr_devc_ser_nbr', 
        prem_nb_col       = 'prem_nb', 
        trsf_pole_nb_col  = 'trsf_pole_nb', 
        outg_rec_nb_col   = 'OUTG_REC_NB'
    )    
    # Below ensures there is only one entry per 'meter' (meter here is defined by a unique grouping of merge_on_mp)
    if any(mp_df.groupby(merge_on_mp).size()>1):
        print('Resolving uniqueness violators')
        mp_df = Utilities_df.resolve_uniqueness_violators(
            df                      = mp_df, 
            groupby_cols            = merge_on_mp, 
            gpby_dropna             = False,
            run_nan_groups_separate = True
        )
    assert(not any(mp_df.groupby(merge_on_mp).size()>1))
else:
    mp_df           = None
    mp_df_cols      = None    

In [ ]:
if not read_dfs_from_pkl:
    start = time.time()
    rcpo_df_OG, ede_typeid_to_reason_df_OG = OutageMdlrPrep.build_reason_counts_per_outage_from_csvs(    
        files_dir                      = files_dir, 
        mp_df                          = mp_df, 
        file_path_glob                 = file_path_glob, 
        file_path_regex                = file_path_regex, 
        min_outg_td_window             = datetime.timedelta(days=days_min_outg_td_window),
        max_outg_td_window             = datetime.timedelta(days=days_max_outg_td_window),
        build_ede_typeid_to_reason_df  = True, 
        batch_size                     = batch_size, 
        cols_and_types_to_convert_dict = None, 
        to_numeric_errors              = 'coerce', 
        assert_all_cols_equal          = assert_all_cols_equal, 
        include_normalize_by_nSNs      = include_normalize_by_nSNs, 
        inclue_zero_counts             = inclue_zero_counts, 
        return_multiindex_outg_reason  = return_multiindex_outg_reason, 
        return_normalized_separately   = return_normalized_separately, 
        verbose                        = True, 
        n_update                       = 1, 
        # grp_by_cols                    = ['trsf_pole_nb_GPD_FOR_SQL', 'no_outg_rec_nb_GPD_FOR_SQL'], 
        grp_by_cols                    = grp_by_col, 
        outg_rec_nb_col                = outg_rec_nb_col,
        trsf_pole_nb_col               = 'trsf_pole_nb', 
        addtnl_dropna_subset_cols      = None, 
        is_no_outage                   = is_no_outage, 
        prem_nb_col                    = 'aep_premise_nb', 
        serial_number_col              = 'serialnumber', 
        include_prem_nbs               = True, 
        set_faulty_mp_vals_to_nan      = False,
        correct_faulty_mp_vals         = False, 
        trust_sql_grouping             = True, 
        drop_gpd_for_sql_appendix      = True, 
        mp_df_cols                     = mp_df_cols, 
        make_all_columns_lowercase     = True
    )
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        rcpo_df_OG.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_OG.pkl'))
        ede_typeid_to_reason_df_OG.to_pickle(os.path.join(save_dir_pkls, f'ede_typeid_to_reason{naming_tag}_df_OG.pkl'))        
else:
    rcpo_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_OG.pkl'))
    ede_typeid_to_reason_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'ede_typeid_to_reason{naming_tag}_df_OG.pkl'))    

In [ ]:
# #TODO DELETE ME!!!!!!!!!
# rcpo_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_OG.pkl'))
# ede_typeid_to_reason_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'ede_typeid_to_reason{naming_tag}_df_OG.pkl'))

In [ ]:
reason_to_ede_typeid_df = AMIEndEvents.invert_ede_typeid_to_reason_df(ede_typeid_to_reason_df_OG)
rcpo_df = rcpo_df_OG.copy()

In [ ]:
time_cols_prefix = 'dummy_lvl_'
#--------------------------------------------------
# Append time information to rcpo_df using either:
#   -  Build no_outg_time_infos_df for baseline data
#   -  Utilize DOVSOutages.append_outg_info_to_df for signal data
#--------------------------------------------------
if is_no_outage:
    # Build no_outg_time_infos_df, which has prem_nbs indices and t_min, t_max (and possible summary_path) columns
    # This is where the time information for each premise number comes from
    paths = Utilities.find_all_paths(base_dir=files_dir, glob_pattern=file_path_glob)
    #-----
    no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
        summary_paths           = [AMIEndEvents.find_summary_file_from_csv(x) for x in paths], 
        output_prem_nbs_col     = 'prem_nbs', 
        output_t_min_col        = 't_min', 
        output_t_max_col        = 't_max', 
        make_addtnl_groupby_idx = True, 
        include_summary_paths   = True
    )
    #-------------------------
    rcpo_df = Utilities_df.merge_dfs(
        df_1        = rcpo_df, 
        df_2        = no_outg_time_infos_df, 
        merge_on_1  = [
            ('index', 'trsf_pole_nb'), 
            ('index', 'no_outg_rec_nb')
        ], 
        merge_on_2  = [
            ('index', 'trsf_pole_nb'), 
            ('index', 'no_outg_rec_nb')
        ], 
        how         = 'left', 
        final_index = 1, 
        dummy_col_levels_prefix = time_cols_prefix
    )
else:
    rcpo_df = DOVSOutages.append_outg_info_to_df(
        df                        = rcpo_df, 
        outg_rec_nb_idfr          = 'index', 
        contstruct_df_args        = None, 
        build_sql_function        = None, 
        build_sql_function_kwargs = None, 
        dummy_col_levels_prefix   = time_cols_prefix
    )
#-------------------------    
assert(rcpo_df.columns.nlevels==2)
time_cols_lvl_0_val = f'{time_cols_prefix}0'
assert(time_cols_lvl_0_val in rcpo_df.columns.get_level_values(0))

In [ ]:
borrow_mp_df_curr_hist_if_exists = True

In [ ]:
df_mp_serial_number_col = 'mfr_devc_ser_nbr'
df_mp_prem_nb_col       = 'prem_nb'
df_mp_install_time_col  = 'inst_ts' 
df_mp_removal_time_col  = 'rmvl_ts' 
df_mp_trsf_pole_nb_col  = 'trsf_pole_nb'

In [ ]:
mp_df_curr_hist = None
if borrow_mp_df_curr_hist_if_exists:
    mp_df_curr_hist = OutageMdlrPrep.try_to_steal_mp_df_curr_hist_from_previous(
        save_dir_base_pkls      = save_dir_base_pkls, 
        days_min_outg_td_window = days_min_outg_td_window, 
        days_max_outg_td_window = days_max_outg_td_window, 
        subdir_regex            = r'^outg_td_window_(\d*)_to_(\d*)_days$', 
        naming_tag              = naming_tag, 
        verbose                 = True, 
    )

In [ ]:
# If borrow_mp_df_curr_hist_if_exists==False OR if True but no appropriate mp_df_curr_hist was found
#   then mp_df_curr_hist must be built!
# In either case, mp_df_curr_hist will be None
if mp_df_curr_hist is None:
    #--------------------------------------------------
    # NOTE: Need mp_df_curr and mp_df_hist separate for functionality, so one cannot simply use mp_df loaded earlier.
    # NOTE: drop_approx_duplicates=False below. These will be dropped later
    #-----
    if grp_by_col=='outg_rec_nb':
        mp_df_curr_hist = DOVSOutages.build_mp_df_curr_hist_for_outgs(
            outg_rec_nbs            = rcpo_df.index.get_level_values(outg_rec_nb_idx_lvl).tolist(), 
            join_curr_hist          = False, 
            addtnl_mp_df_curr_cols  = None, 
            addtnl_mp_df_hist_cols  = None, 
            df_mp_serial_number_col = df_mp_serial_number_col, 
            df_mp_prem_nb_col       = df_mp_prem_nb_col, 
            df_mp_install_time_col  = df_mp_install_time_col, 
            df_mp_removal_time_col  = df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col  = df_mp_trsf_pole_nb_col
        )
    else:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs               = rcpo_df.index.get_level_values(trsf_pole_nbs_idx_lvl).tolist(), 
            join_curr_hist              = False, 
            addtnl_mp_df_curr_cols      = None, 
            addtnl_mp_df_hist_cols      = None, 
            assume_one_xfmr_per_PN      = True, 
            drop_approx_duplicates      = False, 
            drop_approx_duplicates_args = None, 
            df_mp_serial_number_col     = df_mp_serial_number_col, 
            df_mp_prem_nb_col           = df_mp_prem_nb_col, 
            df_mp_install_time_col      = df_mp_install_time_col, 
            df_mp_removal_time_col      = df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col      = df_mp_trsf_pole_nb_col
        )

In [ ]:
#-------------------------
# Make sure all dates are datetime objects, not e.g., strings
mp_df_curr_hist = Utilities_df.ensure_dt_cols(
    df      = mp_df_curr_hist, 
    dt_cols = [df_mp_install_time_col, df_mp_removal_time_col]
)

In [ ]:
#----------------------------------------------------------------------------------------------------
# I don't think I want to do the removal on current, only hist!
# This is because current is used for get_SNs_andor_PNs_for_xfmrs
# e.g., I was missing some PNs because maybe a new meter was installed after rcpo_df['DT_OFF_TS_FULL'].max()
#   So, in all likelihood that was an appropriate meter entry in historical, but this was excluded because
#   there wasn't an entry in current that passed the cuts below
#-----
if is_no_outage:
    t_min_col = 't_min'
    t_max_col = 't_max'
else:
    t_min_col = 'DT_OFF_TS_FULL'
    t_max_col = 'DT_OFF_TS_FULL'
    # t_max_col = 'DT_ON_TS'
#-----    
rcpo_df_t_min = rcpo_df[(time_cols_lvl_0_val, t_min_col)].min()
rcpo_df_t_max = rcpo_df[(time_cols_lvl_0_val, t_max_col)].max()

In [ ]:
mp_df_curr_hist = OutageMdlrPrep.rough_time_slice_and_drop_dups_mp_df_curr_hist(
    mp_df_curr_hist         = mp_df_curr_hist, 
    t_min                   = rcpo_df_t_min, 
    t_max                   = rcpo_df_t_max, 
    df_mp_serial_number_col = 'mfr_devc_ser_nbr', 
    df_mp_prem_nb_col       = 'prem_nb', 
    df_mp_install_time_col  = 'inst_ts', 
    df_mp_removal_time_col  = 'rmvl_ts', 
    df_mp_trsf_pole_nb_col  = 'trsf_pole_nb',
)

In [ ]:
if save_dfs_to_pkl:
    mp_df_curr_hist['mp_df_hist'].to_pickle(os.path.join(save_dir_pkls, f'mp{naming_tag}_df_hist.pkl'))
    mp_df_curr_hist['mp_df_curr'].to_pickle(os.path.join(save_dir_pkls, f'mp{naming_tag}_df_curr.pkl'))

In [ ]:
#--------------------------------------------------
# Need time_infos_df (only if grp_by_col!='outg_rec_nb')
if grp_by_col!='outg_rec_nb':
    if not read_dfs_from_pkl:
        if is_no_outage:
            paths = Utilities.find_all_paths(
                base_dir     = files_dir, 
                glob_pattern = file_path_glob
            )
            #-----
            time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
                summary_paths           = [AMIEndEvents.find_summary_file_from_csv(x) for x in paths], 
                output_prem_nbs_col     = 'prem_nbs', 
                output_t_min_col        = 't_min', 
                output_t_max_col        = 't_max', 
                make_addtnl_groupby_idx = True, 
                include_summary_paths   = False
            )
        else:
            time_infos_df = OutageMdlrPrep.get_outg_time_infos_df(
                rcpo_df                       = rcpo_df, 
                outg_rec_nb_idx_lvl           = outg_rec_nb_idx_lvl, 
                times_relative_to_off_ts_only = True, 
                td_for_left                   = None, 
                td_for_right                  = None
            )
        #-------------------------
        if save_dfs_to_pkl:
            time_infos_df.to_pickle(os.path.join(save_dir_pkls, f'time_infos{naming_tag}_df.pkl'))
    else:
        time_infos_df = pd.read_pickle(os.path.join(save_dir_pkls, f'time_infos{naming_tag}_df.pkl'))

In [ ]:
if not read_dfs_from_pkl:
    start=time.time()
    #--------------------------------------------------
    rcpo_df_raw = MECPODf.project_level_0_columns_from_rcpo_wide(
        rcpo_df_wide = rcpo_df, 
        level_0_val  = 'counts', 
        droplevel    = True
    )
    #-----
    if grp_by_col=='outg_rec_nb':
        rcpo_df_raw = MECPODf.add_outage_active_SNs_to_rcpo_df(
            rcpo_df                    = rcpo_df_raw, 
            set_outage_nSNs            = True, 
            include_outage_premise_nbs = True, 
            df_mp_curr                 = mp_df_curr_hist['mp_df_curr'], 
            df_mp_hist                 = mp_df_curr_hist['mp_df_hist']
        )
        #-----
        rcpo_df_raw = MECPODf.add_active_prim_SNs_to_rcpo_df(
            rcpo_df                             = rcpo_df_raw, 
            direct_SNs_in_outgs_df              = None, 
            outg_rec_nb_col                     = 'index', 
            prim_SNs_col                        = 'direct_serial_numbers', 
            set_prim_nSNs                       = True, 
            sort_SNs                            = True, 
            build_direct_SNs_in_outgs_df_kwargs = {}, 
            mp_df_curr                          = mp_df_curr_hist['mp_df_curr'], 
            mp_df_hist                          = mp_df_curr_hist['mp_df_hist']
        )
    else:
        rcpo_df_raw = OutageMdlrPrep.add_xfmr_active_SNs_to_rcpo_df(
            rcpo_df                                = rcpo_df_raw, 
            trsf_pole_nbs_loc                      = trsf_pole_nbs_loc, 
            set_xfmr_nSNs                          = True, 
            include_active_xfmr_PNs                = True, 
            df_mp_curr                             = mp_df_curr_hist['mp_df_curr'],
            df_mp_hist                             = mp_df_curr_hist['mp_df_hist'], 
            time_infos_df                          = time_infos_df, 
            rcpo_df_to_time_infos_on               = rcpo_df_to_time_infos_on, 
            time_infos_to_rcpo_df_on               = time_infos_to_rcpo_df_on, 
            how                                    = how, 
            rcpo_df_to_PNs_on                      = rcpo_df_to_PNs_on, 
            PNs_to_rcpo_df_on                      = PNs_to_rcpo_df_on, 
            addtnl_get_active_SNs_for_xfmrs_kwargs = dict(
                assert_all_trsf_pole_nbs_found=False
            ), 
            xfmr_SNs_col                           = '_xfmr_SNs', 
            xfmr_nSNs_col                          = '_xfmr_nSNs', 
            xfmr_PNs_col                           = '_xfmr_PNs', 
            xfmr_nPNs_col                          = '_xfmr_nPNs',  
        )
    #-------------------------
    icpo_df_raw = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df                 = rcpo_df_raw, 
        reason_to_ede_typeid_df = reason_to_ede_typeid_df, 
        is_norm                 = False
    )
    #-------------------------
    if save_dfs_to_pkl:
        #-------------------------
        rcpo_df_raw.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_raw.pkl'))
        icpo_df_raw.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_raw.pkl'))

    
    #--------------------------------------------------
    rcpo_df_norm = MECPODf.project_level_0_columns_from_rcpo_wide(
        rcpo_df_wide = rcpo_df, 
        level_0_val  = 'counts_norm', 
        droplevel    = True
    )
    #-------------------------
    icpo_df_norm = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df                 = rcpo_df_norm, 
        reason_to_ede_typeid_df = reason_to_ede_typeid_df, 
        is_norm                 = True, 
        counts_col              = '_nSNs'
    )
    #-------------------------
    if save_dfs_to_pkl:
        #-------------------------
        rcpo_df_norm.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm.pkl'))
        icpo_df_norm.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm.pkl'))

    #--------------------------------------------------
    if grp_by_col=='outg_rec_nb':
        #--------------------------------------------------
        rcpo_df_norm_by_outg_nSNs = MECPODf.build_rcpo_df_norm_by_outg_active_nSNs(
            rcpo_df_raw, 
            df_mp_curr = mp_df_curr_hist['mp_df_curr'], 
            df_mp_hist = mp_df_curr_hist['mp_df_hist']
        )
        #-------------------------
        icpo_df_norm_by_outg_nSNs = MECPODf.convert_rcpo_to_icpo_df(
            rcpo_df                 = rcpo_df_norm_by_outg_nSNs, 
            reason_to_ede_typeid_df = reason_to_ede_typeid_df, 
            is_norm                 = True, 
            counts_col              = '_outg_nSNs'
        )
        #-------------------------
        if save_dfs_to_pkl:
            #-------------------------
            rcpo_df_norm_by_outg_nSNs.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_outg_nSNs.pkl'))
            icpo_df_norm_by_outg_nSNs.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_outg_nSNs.pkl'))
        

        #--------------------------------------------------
        rcpo_df_norm_by_prim_nSNs = MECPODf.build_rcpo_df_norm_by_prim_active_nSNs(
            rcpo_df_raw                         = rcpo_df_raw, 
            direct_SNs_in_outgs_df              = None, 
            outg_rec_nb_col                     = 'index', 
            prim_nSNs_col                       = '_prim_nSNs', 
            prim_SNs_col                        = '_prim_SNs', 
            other_SNs_col_tags_to_ignore        = ['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
            drop_prim_nSNs_eq_0                 = True, 
            new_level_0_val                     = 'counts_norm_by_prim_nSNs', 
            remove_SNs_cols                     = False, 
            build_direct_SNs_in_outgs_df_kwargs = dict(
                equip_typ_nms_of_interest = xfmr_equip_typ_nms_of_interest
            ), 
            df_mp_curr                          = mp_df_curr_hist['mp_df_curr'], 
            df_mp_hist                          = mp_df_curr_hist['mp_df_hist']
        )
        #-------------------------
        icpo_df_norm_by_prim_nSNs = MECPODf.convert_rcpo_to_icpo_df(
            rcpo_df                 = rcpo_df_norm_by_prim_nSNs, 
            reason_to_ede_typeid_df = reason_to_ede_typeid_df, 
            is_norm                 = True, 
            counts_col              = '_prim_nSNs'
        )
        #-------------------------
        if save_dfs_to_pkl:
            #-------------------------
            rcpo_df_norm_by_prim_nSNs.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_prim_nSNs.pkl'))
            icpo_df_norm_by_prim_nSNs.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_prim_nSNs.pkl'))
    else:
        #--------------------------------------------------
        rcpo_df_norm_by_xfmr_nSNs = OutageMdlrPrep.build_rcpo_df_norm_by_xfmr_active_nSNs(
            rcpo_df_raw                            = rcpo_df_raw, 
            trsf_pole_nbs_loc                      = trsf_pole_nbs_loc, 
            xfmr_nSNs_col                          = '_xfmr_nSNs', 
            xfmr_SNs_col                           = '_xfmr_SNs', 
            other_SNs_col_tags_to_ignore           = ['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs'], 
            drop_xfmr_nSNs_eq_0                    = True, 
            new_level_0_val                        = 'counts_norm_by_xfmr_nSNs', 
            remove_SNs_cols                        = False, 
            df_mp_curr                             = mp_df_curr_hist['mp_df_curr'],
            df_mp_hist                             = mp_df_curr_hist['mp_df_hist'], 
            time_infos_df                          = time_infos_df,
            rcpo_df_to_time_infos_on               = rcpo_df_to_time_infos_on, 
            time_infos_to_rcpo_df_on               = time_infos_to_rcpo_df_on, 
            how                                    = how, 
            rcpo_df_to_PNs_on                      = rcpo_df_to_PNs_on, 
            PNs_to_rcpo_df_on                      = PNs_to_rcpo_df_on, 
            addtnl_get_active_SNs_for_xfmrs_kwargs = dict(
                assert_all_trsf_pole_nbs_found=False
            ), 
        )
        #-------------------------
        icpo_df_norm_by_xfmr_nSNs = MECPODf.convert_rcpo_to_icpo_df(
            rcpo_df                 = rcpo_df_norm_by_xfmr_nSNs, 
            reason_to_ede_typeid_df = reason_to_ede_typeid_df, 
            is_norm                 = True, 
            counts_col              = '_xfmr_nSNs'
        )
        #-------------------------
        if save_dfs_to_pkl:
            #-------------------------
            rcpo_df_norm_by_xfmr_nSNs.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))
            icpo_df_norm_by_xfmr_nSNs.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))
    
    #-------------------------
    print(time.time()-start)
    #-------------------------
else:
    rcpo_df_raw               = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_raw.pkl'))
    icpo_df_raw               = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_raw.pkl'))
    #-----
    rcpo_df_norm              = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm.pkl'))
    icpo_df_norm              = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm.pkl'))

    if grp_by_col=='outg_rec_nb':
        rcpo_df_norm_by_outg_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_outg_nSNs.pkl'))
        icpo_df_norm_by_outg_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_outg_nSNs.pkl'))
        #-----
        rcpo_df_norm_by_prim_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_prim_nSNs.pkl'))
        icpo_df_norm_by_prim_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_prim_nSNs.pkl'))
    else:
        rcpo_df_norm_by_xfmr_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))
        icpo_df_norm_by_xfmr_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))

In [ ]:
assert(0)

In [ ]:
group_cols = ['no_outg_rec_nb_GPD_FOR_SQL', 'trsf_pole_nb_GPD_FOR_SQL']
outg_rec_nb_col = 'no_outg_rec_nb'
# files_dir                   = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240620\20230401_20240331\Outages\EvsSums'
files_dir                   = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240621\20230401_20240331\OutgXfmrBaseline\EvsSums'
file_path_glob              = r'events_summary_[0-9]*.csv'
file_path_regex             = None

# batch_size                     = 50
batch_size                     = 10
verbose = True
n_update                       = 1
cols_and_types_to_convert_dict = None
to_numeric_errors              = 'coerce'
make_all_columns_lowercase     = True
assert_all_cols_equal          = True

In [ ]:
evsSum_df_0 = OutageMdlrPrep.build_evsSum_df_from_csvs(
    files_dir                      = files_dir, 
    file_path_glob                 = file_path_glob, 
    file_path_regex                = file_path_regex, 
    batch_size                     = batch_size, 
    cols_and_types_to_convert_dict = cols_and_types_to_convert_dict, 
    to_numeric_errors              = to_numeric_errors, 
    make_all_columns_lowercase     = make_all_columns_lowercase, 
    assert_all_cols_equal          = assert_all_cols_equal,  
    n_update                       = n_update, 
    verbose                        = verbose
)

In [ ]:
evsSum_df                   = evsSum_df_0.copy()
prediction_date             = pd.to_datetime('2023-05-03 00:00:00')
td_min                      = pd.Timedelta('1D')
td_max                      = pd.Timedelta('31D')
data_structure_df           = None
cr_trans_dict               = None
freq                        = '5D'
# freq                        = None
# group_cols                  = ['OUTG_REC_NB_GPD_FOR_SQL', 'trsf_pole_nb_GPD_FOR_SQL']
date_col                    = 'aep_event_dt'

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
## Set to False, False last night, and was much closer to expected (and was non-zero for test case)
# normalize_by_SNs            = True
# normalize_by_time           = True

normalize_by_SNs            = False
normalize_by_time           = False

include_power_down_minus_up = False
regex_patterns_to_remove    = ['.*cleared.*', '.*Test Mode.*']
combine_cpo_df_reasons      = True
xf_meter_cnt_col            = 'xf_meter_cnt'
events_tot_col              = 'events_tot'
# outg_rec_nb_col             = 'outg_rec_nb
trsf_pole_nb_col            = 'trsf_pole_nb'
prem_nb_col                 = 'aep_premise_nb'
serial_number_col           = 'serialnumber'
other_reasons_col           = 'Other Reasons'
total_counts_col            = 'total_counts'
nSNs_col                    = 'nSNs'
trust_sql_grouping          = True
drop_gpd_for_sql_appendix   = True
make_all_columns_lowercase  = True

In [ ]:
evsSum_df = evsSum_df_0.copy()
group_cols = ['no_outg_rec_nb', 'trsf_pole_nb']
evsSum_df = evsSum_df.rename(columns={'no_outg_rec_nb_gpd_for_sql':'no_outg_rec_nb', 'trsf_pole_nb_gpd':'trsf_pole_nb'})

In [ ]:
paths = Utilities.find_all_paths(
    base_dir     = files_dir, 
    glob_pattern = file_path_glob
)
#-----
time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
    summary_paths           = [AMIEndEvents.find_summary_file_from_csv(x) for x in paths], 
    output_prem_nbs_col     = 'prem_nbs', 
    output_t_min_col        = 't_min', 
    output_t_max_col        = 't_max', 
    make_addtnl_groupby_idx = True, 
    include_summary_paths   = False, 
    date_only               = True, 
    date_col                = 'aep_event_dt'
)

In [ ]:
assert(set(group_cols).difference(set(time_infos_df.reset_index().columns.tolist()))==set())
assert(set(group_cols).difference(set(evsSum_df.columns.tolist()))==set())

In [ ]:
time_infos_df

In [ ]:
time_infos_df.loc[('1686036438804', 'Wroux10000')]

In [ ]:
t_min_col = 't_min'
t_max_col = 't_max'

In [ ]:
time_infos_df = time_infos_df.reset_index(drop=False)[group_cols+[t_min_col, t_max_col]].drop_duplicates()
assert((time_infos_df.groupby(group_cols).size()<=1).all())
#-------------------------
evsSum_df_wtinfo = pd.merge(
    evsSum_df, 
    time_infos_df, 
    how      = 'left', 
    left_on  = group_cols, 
    right_on = group_cols
)

In [ ]:
evsSum_df_wtinfo = evsSum_df_wtinfo[
    (evsSum_df_wtinfo[date_col] >= evsSum_df_wtinfo[t_min_col]) & 
    (evsSum_df_wtinfo[date_col] <  evsSum_df_wtinfo[t_max_col])
]

In [ ]:
# In general, this should come from data, NOT be calculated.
evsSum_df_wtinfo['date_relative'] = evsSum_df_wtinfo[t_max_col]-evsSum_df_wtinfo[date_col]

# Shift everything relative to a dummy date
evsSum_df_wtinfo['dummy_date'] = pd.to_datetime('2 Nov. 1987') - evsSum_df_wtinfo['date_relative']

date_col = 'dummy_date'

In [ ]:
evsSum_df = evsSum_df_wtinfo.copy()

# evsSum_df = evsSum_df[(evsSum_df['no_outg_rec_nb']=='Wroux10000') & (evsSum_df['trsf_pole_nb']=='1686036438804')].copy()

In [ ]:
#build_rcpx_from_evsSum_df

#--------------------------------------------------
# Make sure td_min, td_max, and freq are all pd.Timedelta objects
td_min = pd.Timedelta(td_min)
td_max = pd.Timedelta(td_max)
#-------------------------
OutageMdlrPrep.assert_timedelta_is_days(td_min)
OutageMdlrPrep.assert_timedelta_is_days(td_max)
#-----
days_min = td_min.days
days_max = td_max.days
if freq is not None:
    freq   = pd.Timedelta(freq)
    OutageMdlrPrep.assert_timedelta_is_days(freq)
    days_freq = freq.days
#--------------------------------------------------
#-------------------------
prereq_dict = OutageMdlrPrep.perform_build_RCPX_from_end_events_df_prereqs(
    end_events_df                 = evsSum_df, 
    mp_df                         = None, 
    ede_mp_mismatch_threshold_pct = 1.0, 
    grp_by_cols                   = group_cols, 
    outg_rec_nb_col               = outg_rec_nb_col,
    trsf_pole_nb_col              = trsf_pole_nb_col, 
    prem_nb_col                   = prem_nb_col, 
    serial_number_col             = serial_number_col,
    mp_df_cols                    = None,  # Only needed if mp_df is not None
    is_no_outage                  = False, # Only needed if mp_df is not None
    trust_sql_grouping            = trust_sql_grouping, 
    drop_gpd_for_sql_appendix     = drop_gpd_for_sql_appendix, 
    make_all_columns_lowercase    = make_all_columns_lowercase
)
group_cols        = prereq_dict['grp_by_cols']
outg_rec_nb_col   = prereq_dict['outg_rec_nb_col']
trsf_pole_nb_col  = prereq_dict['trsf_pole_nb_col']
prem_nb_col       = prereq_dict['prem_nb_col']
serial_number_col = prereq_dict['serial_number_col']
cols_to_drop      = prereq_dict['cols_to_drop']
rename_cols_dict  = prereq_dict['rename_cols_dict']
mp_df             = prereq_dict['mp_df']
merge_on_ede      = prereq_dict['merge_on_ede']
merge_on_mp       = prereq_dict['merge_on_mp']
#--------------------------------------------------
#-------------------------
if make_all_columns_lowercase:
    date_col          = date_col.lower()
    xf_meter_cnt_col  = xf_meter_cnt_col.lower()
    events_tot_col    = events_tot_col.lower()


#--------------------------------------------------
# Look for any columns ending in _GPD_FOR_SQL and if not included in grpyby cols, then print warning!
found_gpd_for_sql_cols = OutageMdlrPrep.find_gpd_for_sql_cols(
    df        = evsSum_df, 
    col_level = -1
)
if len(found_gpd_for_sql_cols)>0:
    # make loweercase, if needed
    if make_all_columns_lowercase:
        found_gpd_for_sql_cols = [x.lower() for x in found_gpd_for_sql_cols]
    # change names, if needed
    found_gpd_for_sql_cols = [rename_cols_dict.get(x, x) for x in found_gpd_for_sql_cols]
    #-----
    not_included = list(set(found_gpd_for_sql_cols).difference(set(group_cols)))
    if len(not_included)>0:
        print('\n!!!!! WARNING !!!!!\nOutageMdlrPrep.build_rcpx_from_evsSum_df\nFOUND POSSIBLE GROUPBY COLUMNS NOT INCLUDED IN group_cols argument')
        print(f"\tnot_included           = {not_included}\n\tfound_gpd_for_sql_cols = {found_gpd_for_sql_cols}\n\tgroup_cols             = {group_cols}")   
        print('!!!!!!!!!!!!!!!!!!!\n')

#-------------------------
evsSum_df = OutageMdlrPrep.perform_std_col_renames_and_drops(
    end_events_df              = evsSum_df.copy(), 
    cols_to_drop               = cols_to_drop, 
    rename_cols_dict           = rename_cols_dict, 
    make_all_columns_lowercase = make_all_columns_lowercase
)
#-----
nec_cols = group_cols + [date_col, xf_meter_cnt_col, events_tot_col, trsf_pole_nb_col]
assert(set(nec_cols).difference(set(evsSum_df.columns.tolist()))==set()) 

#--------------------------------------------------
# 1. Build rcpo_0
#     Construct rcpx_0 by aggregating evsSum_df by group_cols and by freq on date_col
#--------------------------------------------------
evsSum_df = OutageMdlrPrep.perform_std_col_type_conversions(evsSum_df = evsSum_df)
#-------------------------
if not isinstance(group_cols, list):
    group_cols = [group_cols]
assert(len(set(group_cols).difference(set(evsSum_df.columns.tolist())))==0)
#-------------------------
cr_cols = Utilities.find_in_list_with_regex(
    lst           = evsSum_df.columns.tolist(), 
    regex_pattern = r'cr\d*', 
    ignore_case   = False
)
#-----
# cols_to_drop below are different from cols_to_drop = prereq_dict['cols_to_drop'] above.
#   That defined above is used to remove cases when col_x and f"{col_x}_GPD_FOR_SQL" both exist in the data.
#     For such cases, the trust_sql_grouping parameter settles which is kept.
#   For that defined below, cols_to_drop is used to remove all the columns no longer needed for the analysis
#       Typically, e.g., group_cols = ['outg_rec_nb', 'trsf_pole_nb']
#       ==> cols_to_drop = ['is_first_after_outg', 'aep_opco', 'serialnumber', 'aep_premise_nb']
cols_to_drop = set(evsSum_df.columns.tolist()).difference(
    set(cr_cols+group_cols+[date_col, xf_meter_cnt_col, events_tot_col])
)
cols_to_drop = list(cols_to_drop)
#-------------------------
# Make sure date_col is datetime object
evsSum_df[date_col] = pd.to_datetime(evsSum_df[date_col])

In [ ]:
# evsSum_df[(evsSum_df['no_outg_rec_nb']=='Wroux10000') & (evsSum_df['trsf_pole_nb']=='1686036438804')]

In [ ]:
# evsSum_df.sort_values(by='dummy_date')

In [ ]:
#-------------------------
# All of the cr# columns will be aggregated with np.sum, as will events_tot_col
# xf_meter_cnt_col will be aggregated using np.max
agg_dict = {col:'sum' for col in cr_cols+[events_tot_col, xf_meter_cnt_col]}
agg_dict[xf_meter_cnt_col] = 'max'

In [ ]:
from MECPOCollection import MECPOCollection

In [ ]:
evsSum_df['time_pd_grp']       = np.digitize(evsSum_df['date_relative'].dt.days, np.arange(days_min, days_max+1, days_freq))
evsSum_df['time_pd_grp_width'] = days_freq 

grps_w_widths = list(evsSum_df.groupby(['time_pd_grp', 'time_pd_grp_width']).groups.keys())
# Make sure grps_w_widths is properly sorted
grps_w_widths = natsorted(grps_w_widths, key=lambda x:x[0])
# Also note, for np.digitize, the indices apparently begin at 1, not 0
assert(grps_w_widths[0][0]==1)
grps_w_widths

In [ ]:
rcpx_0 = evsSum_df.drop(columns=cols_to_drop).groupby(group_cols+['time_pd_grp']).agg(agg_dict)

In [ ]:
#--------------------------------------------------
# Using grps_w_widths is a bit overkill currently, since freq is constant.
# However, if freq is allowed to be irregular in the future, the overkill included
#   will be necessary
#-------------------------
time_pds_rename = {}
#-------------------------
# Use padded zeros
#   i.e., instead of '1-6 Days', use '01-06 Days'
#   Will need to know longest string length for this
#-----
# Largest number, therefore largest str, will be days_min + all widths
str_len = len(str(days_min + np.sum([x[1] for x in grps_w_widths])))
#-----
# Looks like a ton of curly braces below, but need double {{ and double }} to escape
# ==> if str_len=2, then str_fmt = '{:02d}'
str_fmt = f'{{:0{str_len}d}}'
#-------------------------
t_rght_i = days_min
# Remember, since np.digitize was used, the indices apparently begin at 1, not 0
for grp_i, width_i in grps_w_widths:
    t_left_i = t_rght_i
    t_rght_i = t_left_i + width_i
    #-----
    rename_i = "{}-{} Days".format(
        str_fmt.format(t_left_i), 
        str_fmt.format(t_rght_i)
    )
    #-----
    assert(grp_i not in time_pds_rename.keys())
    time_pds_rename[grp_i] = rename_i

In [ ]:
#--------------------------------------------------
# 2. Grab meter_cnt_per_gp_srs and all_groups
#--------------------------------------------------
# Project out the meter count per group, as it will be used later
#   This information will be stored in the pd.Series object meter_cnt_per_gp_srs, where the index will
#   contain the group_cols
#-----
# OLD METHOD
# meter_cnt_per_gp_srs = rcpx_0.reset_index()[group_cols+['time_pd_grp', xf_meter_cnt_col]].drop_duplicates().set_index(group_cols+['time_pd_grp']).squeeze()
#-------------------------
meter_cnt_per_gp_srs = rcpx_0.reset_index()[group_cols+['time_pd_grp', xf_meter_cnt_col]].drop_duplicates().set_index(group_cols+['time_pd_grp'])
# Generally, calling .squeeze() in this case works fine, UNLESS meter_cnt_per_gp_srs (which is a pd.DataFrame object as this point)
#   has a single row, in which case a scalar is returned instead of a pd.Series object.
# From the pd.DataFrame.squeeze documentation:
#   Series or DataFrames with a single element are squeezed to a scalar.
# To overcome the issue here, I specifically call .squeeze(axis=1)
# For the more general case, where meter_cnt_per_gp_srs has multiple rows, calling .squeeze(axis=1) should deliver the same results 
#   as calling .squeeze()
# NOTE ALSO: .squeeze(axis=1) could have been tacked on to the above command.  I have broken it out into two steps for illustrative purposes, 
#              so the explanation above makes sense!
meter_cnt_per_gp_srs = meter_cnt_per_gp_srs.squeeze(axis=1)
#-------------------------
assert(meter_cnt_per_gp_srs.shape[0]==meter_cnt_per_gp_srs.index.nunique())
meter_cnt_per_gp_srs.name = nSNs_col

# Will also need the unique groups in rcpx_0
#   This will be used later (see no_events_pd_i below)
#   These can be grabbed from the index of rcpx_0 (excluding the 'time_pd_grp' level)
all_groups = rcpx_0.droplevel('time_pd_grp', axis=0).index.unique().tolist()

In [ ]:
#--------------------------------------------------
# 3. Transform rcpx_0 to the form expected by the model
#     i.e., similar to data_structure_df (if supplied).
#     This is essentially just changing rcpo_0 from long form to wide form
#--------------------------------------------------
#-------------------------
# 3a. Build time_pds_rename
#      Need to convert the time periods, which are currently housed in the 'time_pd_grp' index of 
#        rcpx_0 from their specific dates to the names expected by the model.
#      In rcpx_0, after grouping by the freq intervals, the values of 'time_pd_grp' are equal to the beginning
#        dates of the given interval.
#      These will be converted to the titles contained in final_time_pds below
#      NOTE: This is probably not 100% necessary, but is useful nonetheless
#-------------------------
#--------------------------------------------------
#    Need for data_structure_df?
#     In general, not all curated reasons will be included in the model.
#     Typically, 10 commong curated reasons will be included, and all others will be grouped together in "Other Reasons".
#     Furthermore, some reasons may be combined together, others may be completely removed.
#     For these reasons, it is beneficial to have some sample data (taken from when the model was created) to utilize 
#       in structuring the new data in the same fashion.
#     Additionally, the data will be used to ensure the ordering of columns is correct before the data are fed into 
#       the model.
#--------------------------------------------------
# if data_structure_df is not None:
#     assert(isinstance(data_structure_df, pd.DataFrame) and data_structure_df.shape[0]>0)
#     # final_time_pds should all be found in data_structure_df to help
#     #   ensure the alignment between the current data and data used when modelling
#     assert(set(final_time_pds).difference(data_structure_df.columns.get_level_values(0).unique())==set())

#-------------------------
# 3b. Transform rcpx_0 to the form expected by the model
#      As stated above, this is essentially just changing rcpo_0 from long form to wide form
#      This will probably be formalized further in the future (i.e., function(s) developed to handle)
rename_cols = {
    events_tot_col   : total_counts_col, 
    xf_meter_cnt_col : nSNs_col
}
rcpx_0=rcpx_0.rename(columns=rename_cols)
#-------------------------
total_counts_col = total_counts_col
nSNs_col         = nSNs_col
non_reason_cols  = [nSNs_col, total_counts_col]
#------------------------- 
pd_dfs = []
for grp_i, width_i in grps_w_widths:
    time_pd_i = grp_i
    # Grab the proper time period name from final_time_pd_i
    time_pd_i_rename = time_pds_rename[time_pd_i]
    #-------------------------
    rcpx_0_pd_i = OutageMdlrPrep.project_time_pd_from_rcpx_0_and_prepare(
        rcpx_0                      = rcpx_0, 
        time_pd_i                   = time_pd_i, 
        meter_cnt_per_gp_srs        = meter_cnt_per_gp_srs, 
        all_groups                  = all_groups, 
        non_reason_cols             = non_reason_cols, 
        data_structure_df           = data_structure_df, 
        other_reasons_col           = other_reasons_col, 
        cr_trans_dict               = cr_trans_dict, 
        group_cols                  = group_cols, 
        time_pd_grp_idx             = 'time_pd_grp', 
        time_pd_i_rename            = time_pd_i_rename, 
        regex_patterns_to_remove    = regex_patterns_to_remove, 
        combine_cpo_df_reasons      = combine_cpo_df_reasons, 
        include_power_down_minus_up = include_power_down_minus_up, 
        total_counts_col            = total_counts_col, 
        nSNs_col                    = nSNs_col
    )
    #-------------------------
    # Overkill here (since all time windows are of length freq), but something similar will 
    #   be needed if I want to move to non-uniform period lengths
    # One could, e.g., simply divide by length of freq in days
    if normalize_by_time:
        #-------------------------
        nSNs_b4 = rcpx_0_pd_i[(time_pd_i_rename, nSNs_col)].copy()
        #-----
        rcpx_0_pd_i = MECPODf.normalize_rcpo_df_by_time_interval(
            rcpo_df                 = rcpx_0_pd_i, 
            days_min_outg_td_window = 0, 
            days_max_outg_td_window = width_i, 
            cols_to_adjust          = None, 
            SNs_tags                = None, 
            inplace                 = True
        )
        #-----
        assert(rcpx_0_pd_i[(time_pd_i_rename, nSNs_col)].equals(nSNs_b4))
    #-------------------------
    if normalize_by_SNs:
        cols_to_norm = [x for x in rcpx_0_pd_i.columns if x[1]!=nSNs_col]
        rcpx_0_pd_i[cols_to_norm] = rcpx_0_pd_i[cols_to_norm].divide(rcpx_0_pd_i[(time_pd_i_rename, nSNs_col)], axis=0)
    #-------------------------
    pd_dfs.append(rcpx_0_pd_i)

# Make sure all dfs in pd_dfs look correct
shape_0 = pd_dfs[0].shape
index_0 = pd_dfs[0].index
for i in range(len(pd_dfs)):
    if i==0:
        continue
    assert(pd_dfs[i].shape==shape_0)
    assert(len(set(index_0).symmetric_difference(set(pd_dfs[i].index)))==0)
    #-----
    # Aligning the indices is not strictly necessary, as pd.concat should handle that
    # But, it's best to be safe
    pd_dfs[i] = pd_dfs[i].loc[index_0]

# Build rcpx_final by combining all dfs in pd_dfs
# rcpx_final = pd.concat(pd_dfs, axis=1)
rcpx_final = MECPOCollection.merge_cpo_dfs(
    dfs_coll                      = pd_dfs, 
    max_total_counts              = None, 
    how_max_total_counts          = 'any', 
    SNs_tags                      = MECPODf.std_SNs_cols() + MECPODf.std_nSNs_cols(), 
    cols_to_init_with_empty_lists = MECPODf.std_SNs_cols(), 
    make_cols_equal               = False # b/c if data_structure_df provided, pd_dfs built accordingly
)

#-------------------------
# For the number of SNs per group, use the average number of 
#   SNs for each group across all time periods
nSNs_cols = [x for x in rcpx_final.columns if x[1]==nSNs_col]
#-----
# # There should be one nSNs col for each time period
# assert(len(nSNs_cols)==len(final_time_pds))
#-----
rcpx_final[(nSNs_col, nSNs_col)] = rcpx_final[nSNs_cols].mean(axis=1)
#-----
# Sanity check on the merge
assert(rcpx_final[nSNs_col].notna().all().all())
#-----
# Drop the time-period-specific nSNs columns
rcpx_final = rcpx_final.drop(columns=nSNs_cols)

In [ ]:
rcpx_final

In [ ]:
rcpx_final.loc[('Wroux10000', '1686036438804')]['01-06 Days']

In [ ]:
rcpo_df_raw

In [ ]:
df_1 = rcpo_df_raw.copy()
# df_1 = rcpo_df_norm_by_xfmr_nSNs.copy()
df_2 = rcpx_final.copy()

In [ ]:
df_1 = df_1.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'])

In [ ]:
df_1

In [ ]:
df_2

In [ ]:
# set(df_1.index.get_level_values(0)).symmetric_difference(set(df_2.index.get_level_values(0)))

In [ ]:
df_2b = df_2['01-06 Days'].copy()
df_2b

In [ ]:
df_2b[df_2b['total_counts']>0]

In [ ]:
df_1

In [ ]:
df_1  = df_1.reset_index().sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb'], ignore_index=True, key=natsort_keygen()).set_index(['no_outg_rec_nb', 'trsf_pole_nb'])
df_2b = df_2b.reset_index().sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb'], ignore_index=True, key=natsort_keygen()).set_index(['no_outg_rec_nb', 'trsf_pole_nb'])

In [ ]:
df_1.head()

In [ ]:
df_2b.head()

In [ ]:
df_1.iloc[:, :-8].sum(axis=1)[:10]

In [ ]:
# df_2b.iloc[:,:-1].sum(axis=1)[:10]
df_2b['total_counts'][:10]

In [ ]:
rcpx_final.loc[[('Wroux10000','1686036438804')]]

In [ ]:
ovrlps = list(set(df_2b.index).intersection(df_1.index))
df_1_ovrlp  = df_1.loc[ovrlps]
df_2b_ovrlp = df_2b.loc[ovrlps]

In [ ]:
df_1_ovrlp.iloc[:, :-8].sum(axis=1)[:10]

In [ ]:
# df_2b_ovrlp.iloc[:,:-1].sum(axis=1)[:10]
df_2b_ovrlp['total_counts'][:10]